In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
! pip install langchain transformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 14.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB

In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.prompts import PromptTemplate
from docx import Document
from docx.shared import Inches
import io
import streamlit as st
import torch


def load_llm_with_target_length(target_word_count, prompt_template):
    try:
        model = AutoModelForCausalLM.from_pretrained("thrisha15/llama-2-7b_contentgen", load_in_4bit=True, device_map="auto")
        tokenizer = AutoTokenizer.from_pretrained("thrisha15/llama-2-7b_contentgen")
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None

    if target_word_count:
        target_word_count_int = int(target_word_count)
        effective_word_count = target_word_count_int - 20  # Adjust buffer for potential incompleteness
    else:
        st.error("Please enter a target number of words!")
        return

    # Iterative generation with adjustable max_length
    max_length = 512
    while True:
        # Define the pipeline here
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length,truncation=True)

        # Call the pipeline with keyword arguments
        result = pipe(text_inputs=f"<s>[INST] {prompt_template} [/INST]", max_length=max_length)[0]

        word_count = len(result['generated_text'].split())
        if word_count >= effective_word_count:
            break
        prompt_text = f"<s>[INST] {prompt_template} [/INST] {result['generated_text']}"
        max_length += 512  # Adjust increment as needed

        # Check if exceeding realistic word count
        if max_length > 2048:
            st.error("Target word count too high. Consider reducing or adjusting model parameters.")
            return None

    last_sentence_end = result['generated_text'].rfind('.')
    if last_sentence_end != -1:
        result = result['generated_text'][:last_sentence_end+1]

    return result


def create_word_docx(user_input, paragraph):
    doc = Document()
    doc.add_heading(user_input, level=1)
    doc.add_paragraph(paragraph)
    return doc

st.set_page_config(layout="wide")

def main():
    st.title("Llamagenius Content Synthesizer")

    user_input = st.text_input("Please enter the idea/topic for the article you want to generate!")
    num_words = st.text_input("Minimum Number of words")

    if len(user_input) > 0:
        col1, col2 = st.columns([5, 5])

        with col1:
            st.subheader("Generated Content by our fine_tuned Llama")
            st.write("Topic of the article is: " + user_input)

        if num_words:
            num_words_int = int(num_words)
        else:
            st.error("Please enter a number of words!")
            return

        prompt_template = f"""You are a digital marketing and SEO expert and your task is to write an article on the given topic: {user_input}. The article must be of {num_words_int} words. The article should be lengthy."""

        result = load_llm_with_target_length(target_word_count=num_words, prompt_template=prompt_template)

        if result:
            st.info("Your article has been generated successfully!")
            st.write(result)
        else:
            st.error("Your article couldn't be generated!")

        with col2:
            st.subheader("Final Article to Download")
            if result:
                doc = create_word_docx(user_input, result)
                doc_buffer = io.BytesIO()
                doc.save(doc_buffer)
                doc_buffer.seek(0)

                st.download_button(
                    label='Download Word Document',
                    data=doc_buffer,
                    file_name='document.docx',
                    mime='application/vnd.openxmlformats-officedocument.wordprocessingml.document'
                )

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!ls

app.py	sample_data


In [ ]:

! wget -q -O - ipv4.icanhazip.com

34.143.218.108


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.218.108:8501

npx: installed 22 in 3.264s
your url is: https://fluffy-rockets-throw.loca.lt
2024-04-15 03:59:30.893689: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 03:59:30.893742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 03:59:30.895222: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-15 03:59:32.355893: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The `load_in_4bit` an